<img src="OC.png" width="50" height="50" align="left">    

***

<center> <font size="6"> <span style='color:Blue'> P7: Implémentez un modèle de scoring </span></font> </center>  

***

<font size="3"> <span style="font-size: 1.5em" > **Partie 3 : Analyse du data drift** </span> </font> 

**Contexte**  
**Prêt à dépenser** est une société financière qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.
<img src="pretadepenser.png" width="200" height="200">
**Mission**   
* Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.
* Construire un dashboard interactif à destination des gestionnaires de la relation client permettant d'interpréter les prédictions faites par le modèle, et d’améliorer la connaissance client des chargés de relation client.
* Mettre en production le modèle de scoring de prédiction à l’aide d’une API, ainsi que le dashboard interactif qui appelle l’API pour les prédictions.

Dans ce notebook, nous allons:
* Analyser le data drift avec la librairie Evidently.

# Importation des librairies Python 

In [2]:
import pandas as pd
import numpy as np

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset
from evidently.tests import *

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Importation des données 

In [4]:
reference = pd.read_csv("df_credit_train_sans_imputation_35.csv").iloc[:, 2:]
current = pd.read_csv("df_credit_test_sans_imputation_35.csv").iloc[:, 2:]

In [10]:
print(reference.shape)
reference.head(2)

(206029, 33)


,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,HOUR_APPR_PROCESS_START,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE,BURO_DAYS_CREDIT_MIN,...,PREV_NAME_CONTRACT_STATUS_Approved_MEAN,PREV_NAME_PAYMENT_TYPE_Cashthroughthebank_MEAN,PREV_NAME_TYPE_SUITE_nan_MEAN,PREV_NAME_PRODUCT_TYPE_XNA_MEAN,PREV_NAME_SELLER_INDUSTRY_Consumerelectronics_MEAN,PREV_NAME_YIELD_GROUP_high_MEAN,PREV_NAME_YIELD_GROUP_low_normal_MEAN,REFUSED_DAYS_DECISION_MAX,POS_MONTHS_BALANCE_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_SUM
0,0.003541,-16765,-1186.0,-291,11,0.311267,0.622246,NaN,-828.0,-2586.0,...,1.000000,0.666667,0.000000,0.666667,0.333333,0.000000,0.333333,NaN,-43.785714,NaN
1,0.008019,-19005,-9833.0,-2437,17,NaN,0.650442,NaN,-617.0,NaN,...,0.555556,0.444444,0.555556,0.555556,0.111111,0.222222,0.222222,-181.0,-9.619048,1620000.0


In [11]:
print(current.shape)
current.head(2)

(48744, 33)


,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,HOUR_APPR_PROCESS_START,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE,BURO_DAYS_CREDIT_MIN,...,PREV_NAME_CONTRACT_STATUS_Approved_MEAN,PREV_NAME_PAYMENT_TYPE_Cashthroughthebank_MEAN,PREV_NAME_TYPE_SUITE_nan_MEAN,PREV_NAME_PRODUCT_TYPE_XNA_MEAN,PREV_NAME_SELLER_INDUSTRY_Consumerelectronics_MEAN,PREV_NAME_YIELD_GROUP_high_MEAN,PREV_NAME_YIELD_GROUP_low_normal_MEAN,REFUSED_DAYS_DECISION_MAX,POS_MONTHS_BALANCE_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_SUM
0,0.018850,-19241,-5170.0,-812,18,0.752614,0.789654,0.159520,-1740.0,-1572.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,NaN,-72.555556,NaN
1,0.035792,-18064,-9118.0,-1623,9,0.564990,0.291656,0.432962,0.0,-373.0,...,0.5,0.5,1.0,1.0,0.0,0.5,0.0,NaN,-20.000000,NaN


# Rapport

In [7]:
report = Report(metrics=[
    DataDriftPreset(), 
])

report.run(reference_data=reference, current_data=current)
report

In [12]:
report.save_html("DataDriftReport.html")

# Test suite